<a href="https://colab.research.google.com/github/DanielVered/MathThinkingLabDA/blob/main/Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing & General Info**

In [7]:
import pandas as pd

In [8]:
raw_data = pd.read_excel('FinalResultsRaw.xlsx')

In [18]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10588 entries, 0 to 10587
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   subject                 10588 non-null  object 
 1   step_num                10588 non-null  int64  
 2   uid                     10588 non-null  int64  
 3   trial_start_time        10588 non-null  int64  
 4   rt                      10541 non-null  float64
 5   response                10265 non-null  object 
 6   loop_step               8018 non-null   float64
 7   trial_order             10588 non-null  int64  
 8   variables               10588 non-null  object 
 9   var_mapping             10588 non-null  object 
 10  trial_set               10588 non-null  int64  
 11  trial                   10588 non-null  int64  
 12  core_program            10588 non-null  int64  
 13  variant                 10588 non-null  object 
 14  step                    10588 non-null

In [19]:
raw_data.head(5)

,subject,step_num,uid,trial_start_time,rt,response,loop_step,trial_order,variables,var_mapping,...,loop_type,loop_type_switch,n_iterations,n_loop_lines,updated_var,updated_var_name,step_type,prev_loop_type,correct,same_as_prev_loop_type
0,107B,0,28,25104,2329.0,NaN,NaN,1,"a,i",1=a;2=i,...,NaN,NaN,1,0,a,a,assign,NaN,NaN,0
1,107B,1,29,28013,2395.0,6,0.0,1,"a,i",1=a;2=i,...,for,NaN,2,2,a,a,loop,NaN,0.0,0
2,107B,2,29,29329,805.0,12,1.0,1,"a,i",1=a;2=i,...,for,NaN,2,2,a,a,loop,NaN,0.0,0
3,107B,3,29,31054,1212.0,4,2.0,1,"a,i",1=a;2=i,...,for,NaN,2,2,a,a,loop,NaN,0.0,0
4,107B,4,29,32147,578.0,8,3.0,1,"a,i",1=a;2=i,...,for,NaN,2,2,a,a,loop,NaN,0.0,0
